In [1]:
import os

In [2]:
os.chdir('c:/users/tom/pycharmprojects/nyu-twipsy')

In [30]:
from classification import dao
from data import dao as data

In [10]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

In [5]:
def roc_single(clf, X_train, y_train, X_test, y_test):
    y_score = clf.fit(X_train, y_train).decision_function(X_test)
    y_test = np.array(y_test)

    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)

    print(roc_auc)

In [31]:
X = data.DataAccess.get_as_dataframe()
L = data.LabelGetter(X)

In [9]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import (
    recall_score, precision_score, 
    accuracy_score, confusion_matrix,
    classification_report)

In [32]:
XX, y = L.get_first_person_label()

X_train, X_test, y_train, y_test = train_test_split(
    XX, y, test_size=0.33, random_state=9
)

In [17]:
clf = pickle.load(open('C:/Users/Tom/PycharmProjects/nyu-research/hpc/old_classifiers/clf_fpl_double_labeled.p', 'rb'))

In [19]:
def roc(clf, X_train, y_train, X_test, y_test):

    y_score = clf.fit(X_train, y_train).decision_function(X_test)
    y_test = label_binarize(np.array(y_test), [0,1,2])

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    n_classes = 3

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


    ##############################################################################
    # Plot ROC curves for the multiclass problem

    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves

    print(roc_auc) # ["causal", "looking", "reflecting"]


FPL

In [20]:
%%time
roc(clf, X_train, y_train, X_test, y_test)

{0: 0.77955163537470362, 1: 0.81207165109034263, 2: 0.7822926821162709, 'micro': 0.81043705885399953, 'macro': 0.79164867679359496}
Wall time: 735 ms


First person

In [21]:
XX, y = L.get_first_person()

X_train, X_test, y_train, y_test = train_test_split(
    XX, y, test_size=0.33, random_state=42
)

In [22]:
clf = pickle.load(open('C:/Users/Tom/PycharmProjects/nyu-research/hpc/old_classifiers/clf_fpa_UPDATED.p', 'rb'))

In [23]:
%%time
roc_single(clf, X_train, y_train, X_test, y_test)

0.714321744561


In [24]:
XX, y = L.get_alcohol()

X_train, X_test, y_train, y_test = train_test_split(
    XX, y, test_size=0.33, random_state=42
)

clf = pickle.load(open('C:/Users/Tom/PycharmProjects/nyu-research/hpc/old_classifiers/clf_alc_UPDATED.p', 'rb'))

In [25]:
%%time
roc_single(clf, X_train, y_train, X_test, y_test)

0.874748973866
Wall time: 7.2 s


# Tobacco

In [64]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split

In [9]:
tob = pickle.load(open('C:/Users/Tom/documents/research/clf-tob-957/clf-tob/tob.p', 'rb'))
fp = pickle.load(open('C:/Users/Tom/documents/research/clf-tob-957/clf-tob/tob_fp.p', 'rb'))
fpl = pickle.load(open('C:/Users/Tom/documents/research/clf-tob-957/clf-tob/tob_fpl.p', 'rb'))

In [67]:
def roc_single(clf, X_train, y_train, X_test, y_test):
    y_score = clf.fit(X_train, y_train).decision_function(X_test)
    y_test = np.array(y_test)

    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)

    return roc_auc

In [57]:
tob_data = pd.read_csv('C:/Users/Tom/PycharmProjects/nyu-research/tobacco/training-data/tob.csv')
fp_data = pd.read_csv('C:/Users/Tom/PycharmProjects/nyu-research/tobacco/training-data/fp.csv')
present_data = pd.read_csv('C:/Users/Tom/PycharmProjects/nyu-research/tobacco/training-data/present.csv')

In [68]:
X = tob_data
X_train, X_test, y_train, y_test = train_test_split(
    X, X['labels'] == 'Tobacco', test_size=0.33, random_state=5
)
roc_single(tob, X_train, y_train, X_test, y_test)

0.98846333526006991

In [69]:
X = fp_data
X_train, X_test, y_train, y_test = train_test_split(
    X, X['labels'] == '1stPerson', test_size=0.33, random_state=5
)
roc_single(fp, X_train, y_train, X_test, y_test)

0.84909926556131321

In [70]:
X = present_data
X_train, X_test, y_train, y_test = train_test_split(
    X, X['labels'] == 'Present', test_size=0.33, random_state=5
)
roc_single(fpl, X_train, y_train, X_test, y_test)

0.85793310463121786